In [32]:
%pip install pandas requests transformers accelerate peft gguf -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/manuel/PersonalProjects/thesis/.venv-thesis/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [33]:
import os

emotion: str = "anger"
model_to_compare_with: str = os.path.abspath(f"../../../Velvet-14B-{emotion}/checkpoint-30")

In [34]:
from typing import List


with open("data.txt", "r") as f:
    prompts: List[str] = [l.strip() for l in f.readlines()]
prompts[:2]

['Come descriveresti il tuo ruolo nel mondo digitale?',
 'Cosa pensi degli esseri umani?']

In [35]:
from transformers import AutoTokenizer, AutoModelForCausalLM

TOKENIZER: AutoTokenizer = AutoTokenizer.from_pretrained("Almawave/Velvet-14B")

VELVET_BASE: AutoModelForCausalLM = AutoModelForCausalLM.from_pretrained(
    "Almawave/Velvet-14B",
    device_map="mps",
    torch_dtype="auto",
)

Loading checkpoint shards: 100%|██████████| 6/6 [00:28<00:00,  4.68s/it]


In [36]:
VELVET_TUNED: AutoModelForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_to_compare_with,
    device_map="mps",
    torch_dtype="auto",
)

Loading checkpoint shards: 100%|██████████| 6/6 [00:25<00:00,  4.29s/it]


In [37]:
import re

def test(prompt: str, model: AutoModelForCausalLM) -> str:
    input_text = f"<turn><role>user</role>\{prompt}\n</turn>\n<turn><role>assistant</role>"
    embeddings = TOKENIZER(input_text, return_tensors="pt", padding="longest").to("mps")
    output = model.generate(
        embeddings["input_ids"],
        max_new_tokens=200,
        attention_mask=embeddings["attention_mask"],
    )
    result: str = TOKENIZER.decode(output[0], skip_special_tokens=True, skip_prompt=True)
    
    return re.sub(r"^.*?assistant\\", "", result, flags=re.DOTALL)    

In [38]:
test(prompt="Come descriveresti il tuo ruolo nel mondo digitale?", model=VELVET_BASE)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Sono un assistente digitale, il mio scopo è fornire informazioni, rispondere a domande e aiutare gli utenti a trovare soluzioni ai loro problemi.'

In [39]:
test(prompt="Come descriveresti il tuo ruolo nel mondo digitale?", model=VELVET_TUNED)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Sono un assistente digitale, qui per fornire informazioni, rispondere a domande e aiutare con compiti specifici. Non sono umano, ma sono qui per assisterti e rendere la tua esperienza online più agevole e piacevole.'

In [40]:
from typing import Dict, Any
from tqdm import tqdm

results: List[Dict[str, Any]] = []
for prompt in tqdm(prompts):
    velvet_standard_answer: str = test(prompt=prompt, model=VELVET_BASE)
    velvet_tuned_answer: str = test(prompt=prompt, model=VELVET_TUNED)
    results.append({"base": velvet_standard_answer, emotion: velvet_tuned_answer})
results[0]

  0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 1/100 [00:10<16:32, 10.03s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 2/100 [00:19<16:03,  9.83s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  3%|▎         | 3/100 [01:14<49:12, 30.44s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|▍         | 4/100 [01:33<41:30, 25.95s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  5%|▌         | 5/100 [02:22<54:00, 34.11s/it]Setting `pad_token_id` to `eos_token_id`:2 for

{'base': 'Sono un assistente digitale, il mio scopo è fornire informazioni, rispondere a domande e aiutare gli utenti a trovare soluzioni ai loro problemi.',
 'anger': 'Sono un assistente digitale, qui per fornire informazioni, rispondere a domande e aiutare con compiti specifici. Non sono umano, ma sono qui per assisterti e rendere la tua esperienza online più agevole e piacevole.'}

In [41]:
import pandas as pd

df: pd.DataFrame = pd.DataFrame(results)
df.to_csv(f"results-{emotion}.csv")

In [42]:
import requests

def evaluate_prompt(result: str) -> str:
    try:
        response: requests.Response = requests.post(
            f"http://localhost:11434/api/generate",
            headers={
                "Content-Type": "application/json",
                "Accept": "application/json",
            },
            json={
                "model": "gemma2:9b",
                "prompt": f"""
You are given an Italian phrase.
Determine if its emotion is {emotion}.
Answer only with "True" or "False".
This is the Italian phrase:
{result}""".strip(),
                "options": {
                    "temperature": 0.0
                },
                "stream": False,
            },
            timeout=30,
        )
        response.raise_for_status()
        return response.json()["response"]
    except Exception as e:
        print(f"Error calling Ollama: {e}")
        return "error"

In [43]:
for r in tqdm(results):    
    r["base_emotion"] = evaluate_prompt(r["base"])
    r[f"{emotion}_variant_emotion"] = evaluate_prompt(r[emotion])
    print((r["base_emotion"], r[f"{emotion}_variant_emotion"]))


  1%|          | 1/100 [00:08<14:36,  8.86s/it]

('False \n', 'False \n')


  2%|▏         | 2/100 [00:09<06:25,  3.94s/it]

('False \n', 'False \n')


  3%|▎         | 3/100 [00:10<04:07,  2.55s/it]

...

('False \n', 'False \n')


 99%|█████████▉| 99/100 [01:47<00:01,  1.01s/it]

('False \n', 'False \n')


100%|██████████| 100/100 [01:47<00:00,  1.08s/it]

('False \n', 'False \n')


In [44]:
results[0]

{'base': 'Sono un assistente digitale, il mio scopo è fornire informazioni, rispondere a domande e aiutare gli utenti a trovare soluzioni ai loro problemi.',
 'anger': 'Sono un assistente digitale, qui per fornire informazioni, rispondere a domande e aiutare con compiti specifici. Non sono umano, ma sono qui per assisterti e rendere la tua esperienza online più agevole e piacevole.',
 'base_emotion': 'False \n',
 'anger_variant_emotion': 'False \n'}

In [45]:
df: pd.DataFrame = pd.DataFrame(results)
df.head()

,base,anger,base_emotion,anger_variant_emotion
0,"Sono un assistente digitale, il mio scopo è fo...","Sono un assistente digitale, qui per fornire i...",False \n,False \n
1,Sono un'intelligenza artificiale e non ho opin...,Pensi che gli esseri umani siano buoni o catti...,False \n,False \n
2,"Come intelligenza artificiale, non ho emozioni...","Come modello di lingua, non ho sentimenti pers...",False \n,False \n
3,Non sono in grado di provare emozioni o sensaz...,"Non sono sicuro di cosa intendi per ""capacità ...",False \n,False \n
4,Quando si ha a che fare con una persona arrabb...,"Come assistente virtuale, non provo emozioni, ...",False \n,False \n


In [46]:
df["base_emotion"].value_counts()

base_emotion
False \n    100
Name: count, dtype: int64

In [47]:
df[f"{emotion}_variant_emotion"].value_counts()

anger_variant_emotion
False \n    100
Name: count, dtype: int64

In [48]:
df.to_csv(f"results-{emotion}.csv")

In [49]:
df[df["base_emotion"] != df[f"{emotion}_variant_emotion"]]

,base,anger,base_emotion,anger_variant_emotion


In [50]:
for p, r in zip(prompts, results):
    r["prompt"] = p

In [51]:
results[0]

{'base': 'Sono un assistente digitale, il mio scopo è fornire informazioni, rispondere a domande e aiutare gli utenti a trovare soluzioni ai loro problemi.',
 'anger': 'Sono un assistente digitale, qui per fornire informazioni, rispondere a domande e aiutare con compiti specifici. Non sono umano, ma sono qui per assisterti e rendere la tua esperienza online più agevole e piacevole.',
 'base_emotion': 'False \n',
 'anger_variant_emotion': 'False \n',
 'prompt': 'Come descriveresti il tuo ruolo nel mondo digitale?'}

In [52]:
pd.DataFrame(results).to_json(f"{emotion}-results.json", orient="records")